In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error

In [26]:
data = pd.read_csv('TRAIN.csv')

In [29]:
test = pd.read_csv('TEST_FINAL.csv')

In [30]:
data['Discount'].replace({'Yes':1,'No':0},inplace=True)

In [31]:
data.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,1,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,1,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,1,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,1,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,1,62,45614.52


In [32]:
data2 = pd.concat([data.Store_id,data.Holiday,data.Discount,data.Sales],axis=1)

In [7]:
test_data = test.drop(columns=['ID','Store_Type','Location_Type','Region_Code','Date'])

In [8]:
test_data['Discount'].replace({'Yes':1,'No':0},inplace=True)  ## Replace Yes to 0 ans No to 1

In [9]:
test_data2 = test_data

In [109]:
def get_data(st,en,df,y,sum30,sum20,sum10):
    ''' This Function is used to calculate avg and last 5 days sales and then convert them to pandas dataframme
    Input - st - Starting Index
          en - Last Index
          df - dataframe
          y - Sales
          sum30 - sum of sales of last 30 days
          sum20 - sum of sales of last 20 days
          sum10 - sum of sales of last 10 days
    returns data and averages '''      

    
    df = df.iloc[st:en]
    y = np.asarray(y)                  
   
    sz = en-st                    ## Size of Dataset
    col_name = ['t1','t2','t3','t4','t5','av30','av20','av10']
    for col in col_name : df[col] = np.zeros((sz,1))
  
    ycopy = np.zeros((sz+30,1))        ## Ycopy used for storing predictions extra 30 size is used to store values of last 30 days
    ycopy[:30] = y[st-30:st].reshape(30,1)
    
    

    df['t1'][st] = ycopy[29] 
    df['t2'][st] = ycopy[28]
    df['t3'][st] = ycopy[27]
    df['t4'][st] = ycopy[26] 
    df['t5'][st] = ycopy[25]
    df['av30'][st] = sum30/30.0   ### Calculating Averages
    df['av20'][st] = sum20/20.0
    df['av10'][st] = sum10/10.0
 
    ## Predicting Sales day wise 
    for i in range(sz-1):
        ycopy[30+i] = clf.predict(np.asarray(df.iloc[i]).reshape(1,10))
        
        df['t1'][st+i+1] = ycopy[30+i]       ## Storing last 5 days sales and averages 
        df['t2'][st+i+1] = ycopy[29+i]
        df['t3'][st+i+1] = ycopy[28+i]
        df['t4'][st+i+1] = ycopy[27+i]
        df['t5'][st+i+1] = ycopy[26+i]
        sum30 += (ycopy[30+i]-ycopy[i])
        sum20 += (ycopy[30+i]-ycopy[i+10])
        sum10 += (ycopy[30+i]-ycopy[i+20])
        df['av30'][st+i+1] = sum30/30.0
        df['av20'][st+i+1] = sum20/20.0
        df['av10'][st+i+1] = sum10/10.0

    return df,y[st:en],sum30,sum20,sum10    


In [110]:
def get_test_data(df,y,sum30,sum20,sum10):
    ''' This function is used to calculate prediction on test data and return output and indexs of the data 
    Input Y - Last 30 Days Sales from given training data '''

    df = df.reset_index()
    idx = df['index']
    df.drop(columns=['index','Store_id'],inplace=True)   ## Dropping columns which are of no use
    sz = len(df)
    col_name = ['t1','t2','t3','t4','t5','av30','av20','av10']
    for col in col_name : df[col] = np.zeros((sz,1))          ## Initializing
  
   
    ycopy = np.zeros((sz+30,1))    ## Storing values of last 30 days
    ycopy[:30] = y

    df['t1'][0] = ycopy[29]
    df['t2'][0] = ycopy[28]
    df['t3'][0] = ycopy[27]
    df['t4'][0] = ycopy[26]
    df['t5'][0] = ycopy[25]
    df['av30'][0] = sum30/30.0   ## Calculating Averages
    df['av20'][0] = sum20/20.0
    df['av10'][0] = sum10/10.0
    
    for i in range(sz-1):
        
        ycopy[30+i] = clf.predict(np.asarray(df.iloc[i]).reshape(1,10))
    
        df['t1'][i+1] = ycopy[30+i]    ## Storing and calculating new averages and values
        df['t2'][i+1] = ycopy[29+i]
        df['t3'][i+1] = ycopy[28+i]
        df['t4'][i+1] = ycopy[27+i]
        df['t5'][i+1] = ycopy[26+i]
        sum30 += ((ycopy[30+i]-ycopy[i]))
        sum20 += (ycopy[30+i]-ycopy[i+10])
        sum10 += (ycopy[30+i]-ycopy[i+20])
        df['av30'][i+1] = sum30/30.0
        df['av20'][i+1] = sum20/20.0
        df['av10'][i+1] = sum10/10.0
        
    return df,idx   


In [ ]:
clf = RandomForestRegressor(n_estimators=100,max_depth=15)
days = []
days = [151,212,273,334,396,455,516] ## Training for 151 days then predicting for next 61 days then using these 61 days
                                     ### predicting sales for next 2 months
y_pred = np.zeros((len(test_data),))
for i in tqdm(range(1,366)):
    df = data2[data2.Store_id==i]
    df = df.reset_index()
    y = df.Sales
    y1 = y
    df.drop(columns=['index','Sales','Store_id'],inplace=True)
    nd = 30                  ## Initial no of days which will be not used as avg cannot be calculated for these rows
    lent = days[0]- nd       ## length of intial data
    sum30 = y[:30].sum()     ## Calculating sum of last 30,20,10,days
    sum20 = y[10:30].sum()
    sum10 = y[20:30].sum()
    
    dfcopy = df.iloc[nd:nd+lent]  ## Locating Data which will be used
    
    col_name = ['t1','t2','t3','t4','t5','av30','av20','av10']
  
      
    for col in col_name : dfcopy[col] = np.zeros((lent,1))  
   
    for j in range(nd,nd+lent):
        dfcopy['t1'][j-nd] = y1[j-1]       ## Calculating averages and last 5 days sales and storing them
        dfcopy['t2'][j-nd] = y1[j-2]
        dfcopy['t3'][j-nd] = y1[j-3]
        dfcopy['t4'][j-nd] = y1[j-4]
        dfcopy['t5'][j-nd] = y1[j-5]
        dfcopy['av30'][j-nd] = sum30/30.0
        dfcopy['av20'][j-nd] = sum20/20.0
        dfcopy['av10'][j-nd] = sum10/10.0
        sum30 += (y1[j]-y1[j-nd])
        sum20 += (y1[j]-y1[j-nd+10])
        sum10 += (y1[j]-y1[j-nd+20])
        
       
    y1 = y1[nd:nd+lent]      ## Sales which will be used 
    
    clf.fit(dfcopy,y1)      ## Training For the first Time

    full_data = dfcopy     ## Data Used Till Now

    for j in range(len(days)-1):
      df3,yloc,sum30,sum20,sum10 = get_data(days[j],days[j+1],df,y,sum30,sum20,sum10)  ## Predicting values on next 2 Months
      ypred = clf.predict(df3)                       ## Predicting 
      print(i,mean_squared_log_error(yloc,ypred))
      full_data = pd.concat([full_data,df3],axis=0)   ### Now Add Initial data and Predicted data 
      full_y = y[nd:days[j+1]]             
      clf.fit(full_data,full_y)                       ### train model for this new data

    ### Training done for Store i
    ### Prediction on Test Data

    testd2 = test_data2[test_data2.Store_id==i]
    testy = np.asarray(y[516-30:]).reshape(30,1)
    df3,idx = get_test_data(testd2,testy,sum30,sum20,sum10)
    y_out = clf.predict(df3)
    y_pred[idx] = y_out    ### Storing Values in Final Output
   

In [ ]:
### Converting to CSV
ans = pd.DataFrame()
ans['ID'] = test['ID']
ans['Sales'] = y_pred
ans.to_csv('final,csv',index=false)